### This notebook evaluates a grid run of multiple models using pre-defined metrics 

In [22]:
import numpy as np
import pandas as pd
import os
import scipy.stats 
import h5py
import loss
from tensorflow.python.keras import backend as K

In [2]:
from modelzoo import *

In [17]:
def np_mse(a, b):
    return ((a - b)**2).mean()

In [3]:
grid_dir = 'datasets/ATAC_bin/grid4'
preds = [os.path.join(grid_dir, file) for file in os.listdir(grid_dir) if (file.endswith('.h5') and 'pred_' in file)]

In [4]:
pred = preds[0]

In [5]:
dataset = h5py.File(pred, 'r')

In [6]:
test_pred = dataset['test_pred'][:]
test_y = dataset['test_y'][:]

In [18]:
mse = np_mse(test_y, test_pred)

In [77]:
scipy_pr(test_y[:,:,:], test_pred[:,:,:])

array([0.65843824, 0.57946038, 0.67114482, 0.66405675, 0.65058335,
       0.64318487, 0.64461434, 0.59419452, 0.62060418, 0.64374626,
       0.63448487, 0.65220019, 0.65092853, 0.60915475, 0.62014465,
       0.63481027, 0.64856279, 0.63178551, 0.61218004, 0.64591851,
       0.59147649, 0.60651696, 0.60762309, 0.58107284])

In [56]:
def scipy_pr(y_true, y_pred):
    _,_,T = y_true.shape
    res_pr = np.zeros(T)
    for t in range(T):
        pr = scipy.stats.pearsonr(y_true[:,:,t].flatten(), y_pred[:,:,t].flatten())[0]
        res_pr[t] = pr
    return res_pr

In [87]:
def np_poiss(y_true, y_pred):
    return np.nanmean(y_pred - y_true * np.log(y_pred))

In [85]:
def loop_pois(y_true, y_pred):
    _,_,T = y_true.shape
    res_pr = np.zeros(T)
    for t in range(T):
        pr = np_poiss(y_true[:,:,t].flatten(), y_pred[:,:,t].flatten())
        res_pr[t] = np.nanmean(pr) 
#         print(pr)
#         break
    return res_pr

In [88]:
def get_metrics(y_true, y_pred):
    mse = np_mse(test_y, test_pred)
    pr = scipy_pr(test_y, test_pred)
    pois = np_poiss(test_y, test_pred)
    return ({'mse': mse,
            'pr': pr,
             'pois': pois})

In [ ]:
eval_metrics = {}
for pred in preds:
    dataset = h5py.File(pred, 'r')
    test_pred = dataset['test_pred'][:]
    test_y = dataset['test_y'][:]
    eval_metrics[pred] = get_metrics(test_y, test_pred)

/home/shush/tf_2/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
/home/shush/tf_2/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/home/shush/tf_2/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in multiply
  
